In [ ]:
#install required packages
!pip install -q transformers peft  accelerate bitsandbytes safetensors sentencepiece chromadb langchain sentence-transformers gradio git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# import dependencies
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig, pipeline
from peft import PeftModel

import os
import sys
import gradio as gr
from google.colab import drive

import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
# mount google drive and specify folder path
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# specify model huggingface mode name
base_model = "openthaigpt/openthaigpt-1.0.0-beta-7b-chat-ckpt-hf"

lora_weights='/content/drive/MyDrive/My LLM Lab/TrainModel/ThaiNutriChatNew-500'

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model)
tokenizer.bos_token_id = 1  # Set beginning of sentence token id

# specify stop token ids
stop_token_ids = [0]

tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="auto"
)

# if lora_weights:
#       model = PeftModel.from_pretrained(
#           model,
#           lora_weights,
#           torch_dtype=torch.float16,
#       )

# unwind broken decapoda-research config
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "intfloat/multilingual-e5-small"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [ ]:
# load from disk
persist_directory='/content/drive/MyDrive/My LLM Lab/chroma_db'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# specify the retriever
retriever = vectordb.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8})

In [ ]:
from transformers import GenerationConfig

config = GenerationConfig(
          do_sample=True,
          top_k=5,
          temperature=0.1,
          repetition_penalty=1.2,
    )

In [ ]:
pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=2048,
        generation_config=config,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        framework='pt'
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipe)

The model 'OptimizedModule' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCaus

In [ ]:
from langchain import PromptTemplate

# template = """คุณเป็นผู้ช่วยตอบคำถาม ตอบคำถามตามความเป็นจริงและเป็นประโยชน์มากที่สุด ใช้ context ต่อไปนี้เพื่อตอบ question ในตอนท้าย โดยตอบแค่ ใช่ หรือ ไม่ใช่เท่านั้น ถ้าไม่รู้คำตอบก็บอกว่าไม่รู้ ไม่ต้องพยายามสร้างคำตอบ
# {context}

# Question: {question}
# Helpful Answer:
# """

template = """Use the following pieces of context to answer the question at the end. answer only yes or no. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}

Question: {question}
Helpful Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
# build conversational retrieval chain with memory (rag) using langchain
def create_conversation(query: str, chat_history: list) -> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )

        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            verbose=True,
            memory=memory,
            # combine_docs_chain_kwargs={"prompt": prompt},
            get_chat_history=lambda h: h,
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history


    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

In [ ]:
# build gradio ui
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # ThaiNutriChat

        """
    )

    chatbot = gr.Chatbot(height=240)
    msg     = gr.Textbox(label="Prompt")
    btn     = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(fn=create_conversation, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(fn=create_conversation, inputs=[msg, chatbot], outputs=[msg, chatbot])
gr.close_all()

demo.launch(server_name="0.0.0.0", share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://94719a4831c7559bc9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

5. ถาม การกินใยอาหารสามารถช่วยลดระดับไขมันในเลือดได้จริงหรือไม่
ตอบ ใยอาหารมีอยู่ในผัก ผลไม้และธัญพืชไม่ขัดสีซึ่งใยอาหารมี2 ชนิด ได้แก่ ใยอาหารชนิดละลายน้ําและ
ชนิดไม่ละลายน้ํา โดยใยอาหารชนิดไม่ละลายน้ํามีประโยชน์เพิ่มเนื้ออุจจาระ ป้องกันและลดอาการ
ท้องผูก ส่วนใยอาหารชนิดละลายน้ํามีประโยชน์ลดการดูดซึมไขมันในอาหาร ทําให้สามารถลดระดับ
ไขมันในเลือดได้โดยอาหารที่มีใยอาหารชนิดละลายน้ําสูง ได้แก่ข้าวกล้อง ข้าวโพด ลูกเดือย แอปเปิ้ล
ฝรั่ง ข้าวโพดอ่อน เป็นต้น

4. ถาม การเลือกกินอาหารที่ไม่มีคอเลสเทอรอล จะไม่ทําให้ไขมันในเลือดสูง จริงหรือไม่
ตอบ การกินอาหารที่ไม่มีคอเลสเทอรอลหรือมีคอเลสเทอรอลน้อยเพียงอย่างเดียว ไม่สามารถควบคุมระดับ
ไขมันในเลือดได้ เนื่องจากการกินอาหารเพื่อควบคุมระดับไขมันในเลือดนั้นต้องคํานึงถึงปริมาณไขมัน
ทั้งหมด

/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: ไข่เป็ด ให้พลังงานมากกว่า ไข่ไก่ ใช่หรือ ไม่
Helpful Answer:

> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

การควบคุมปริมาณไขมันในอาหาร ทำได้โดยการเลือกกินหรือปรุงเนื้อสัตว์ หรือผลิตภัณฑ์จากสัตว์
ที่มีไขมันต่ำ เช่น เนื้อไก่ไม่ติดหนัง เนื้อหมูไม่ติดมัน นมไขมันต่ำหรือพร่องมันเนย โยเกิร์ตไขมันต่ำ ร่วมกับ
การปรุงอาหารด้วยน้ำมันในปริมาณที่แนะนำ หรือเลือกกินอาหารที่ปรุงด้วยวิธีการที่ใช้น้ำมันน้อย ได้แก่ ต้ม ยำ
ย่างอบโดยหากมีการกินถั่วเปลือกแข็งกะทิครีมเทียมถื

...


> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

5. ออกกําลังกายหรือเคลื่อนไหวร่างกายอย่างสม่ําเสมอ
ออกกําลังกายติดต่อกันอย่างสม่ําเสมอคือการออกกําลังกายทุกวันหรือวันเว้นวันหรืออย่างน้อย
สัปดาห์ละ 3 ครั้งๆ ละ 30-45 นาทีส่งผลให้ไขมันเปลี่ยนแปลงไปในทางที่ดีขึ้น โดยสามารถลดระดับ
คอเลสเทอรอลและไตรกลีเซอไรด์ รวมถึงเพิ่มระดับเอชดีเอล (ไขมันที่ดี) นอกจากนี้การออกกําลังกาย
ยังเพิ่มสมรรถภาพการทํางานของปอดและหัวใจ ได้แก่ การเดิน วิ่ง จ๊อกกิ้ง เต้นรํา ขี่จักรยาน ว่ายน้ํา
เต้นแอโรบิก รํามวยจีน รํากระบอง เป็นต้น

โรคไขมันในเลือดสูง คือ การที่ร่างกายมีระดับไขมันในเลือดอยู่ในเกณฑ์ผิดปกติได้แก่ การมีปริมาณ
คอเลสเทอรอลรวม ไตรกลีเซอไรด์หรือแอลดีแอล(LDL) คอเลสเทอรอลในระดับสูง หรือมีระดับเอชดีแอล(HDL)
คอเลสเทอรอลตำ โดยปริมาณคอเลสเทอรอลรวม คอเลสเทอรอลช

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

การควบคุมปริมาณไขมันในอาหาร ทำได้โดยการเลือกกินหรือปรุงเนื้อสัตว์ หรือผลิตภัณฑ์จากสัตว์
ที่มีไขมันต่ำ เช่น เนื้อไก่ไม่ติดหนัง เนื้อหมูไม่ติดมัน นมไขมันต่ำหรือพร่องมันเนย โยเกิร์ตไขมันต่ำ ร่วมกับ
การปรุงอาหารด้วยน้ำมันในปริมาณที่แนะนำ หรือเลือกกินอาหารที่ปรุงด้วยวิธีการที่ใช้น้ำมันน้อย ได้แก่ ต้ม ยำ
ย่างอบโดยหากมีการกินถั่วเปลือกแข็งกะทิครีมเทียมถือเป็นอาหารในหมวดไขมันจะต้องมีการลดปริมาณน้ำมัน
ที่ใช้ในการประกอบอาหารด้วย

ข้อแนะนำ ปริมาณคอเลสเทอรอลที่ควรบริโภค คือ ผู้ที่มีภาวะไขมันในเลือดสูงควรบริโภคคอเลสเทอรอล
ไม่เกิน200 มิลลิกรัมต่อวัน คนปกติทั่วไปควรบริโภคคอเลสเทอรอลไม่เกิน300 มิลลิกรัมต่อวัน

5. ออกกําลังกายหรือเคลื่อนไหวร่างกายอย่างสม่ําเสมอ
ออกกําลังกายติดต่อกันอย่างสม่ําเสมอคือก

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://94719a4831c7559bc9.gradio.live
